# Setup

In [1]:
from dotenv import load_dotenv

In [2]:
_ = load_dotenv()

In [3]:
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=cdiD-9MMpb0"

# First Simple Chain

In [4]:
from langchain_openai.chat_models import ChatOpenAI

In [5]:
# OPENAI_API_KEY is automatically sourced
model = ChatOpenAI(model="gpt-3.5-turbo")

# Example of query
model.invoke("What MLB team won the World Series during the COVID-19 pandemic?")

AIMessage(content='The Los Angeles Dodgers won the World Series during the COVID-19 pandemic in 2020.', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 21, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-894f9501-5c2c-435a-a79f-7503da8f08f6-0', usage_metadata={'input_tokens': 21, 'output_tokens': 19, 'total_tokens': 40})

# Output Parser

In [6]:
from langchain_core.output_parsers import StrOutputParser

In [7]:
parser = StrOutputParser()

# 2-steps chain
chain = model | parser

chain.invoke("What MLB team won the World Series during the COVID-19 pandemic?")

'The Los Angeles Dodgers won the World Series during the COVID-19 pandemic in 2020.'

# Chat Prompt Template

In [8]:
from langchain.prompts import ChatPromptTemplate

In [9]:
template = """
Answer the question based on the context below. If you can't \
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

print(template)

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Mary's sister is Susana.", question="Who is Mary's sister?")


Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}



'Human: \nAnswer the question based on the context below. If you can\'t answer the question, reply "I don\'t know".\n\nContext: Mary\'s sister is Susana.\n\nQuestion: Who is Mary\'s sister?\n'

In [10]:
# 3-steps chain
chain = prompt | model | parser

chain.invoke({
    "context": "Mary's sister is Susana",
    "question": "Who is Mary's sister?"
})

'Susana'

# Combining Chains

In [11]:
translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

In [12]:
from operator import itemgetter

In [13]:
translation_chain = (
    {"answer": chain, "language": itemgetter("language")}
    | translation_prompt
    | model
    | parser
)

In [14]:
# # WEIRD BUG, SEE LATER...
# translation_chain.invoke(
#     {
#         "context": "Mary's sister is Susana. She doesn't have any more siblings.",
#         "question": "How many sisters does Mary have?",
#         "language": "German",
#     }
# )

# Transcribing the YouTube Video

In [15]:
import os
import tempfile

import whisper
from pytube import YouTube

> **NOTE**
> 
> In order to run the following script, you need to have the `ffmpeg` library installed on your OS, which, on Linux, will require:
> ```bash
> sudo apt update
> sudo apt install ffmpeg
> ```

In [17]:
if not os.path.exists("transcription.txt"):
    youtube = YouTube(YOUTUBE_VIDEO)
    audio = youtube.streams.filter(only_audio=True).first()

    # Let's load the base model. This is not the most accurate
    # model but it's fast.
    whisper_model = whisper.load_model("base")

    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()

        with open("transcription.txt", "w") as file:
            file.write(transcription)

# Using the Entire Transcription as Context

In [18]:
with open("transcription.txt") as file:
    transcription = file.read()

transcription[:100]

"I think it's possible that physics has exploits and we should be trying to find them. arranging some"

In [19]:
try:
    chain.invoke({
        "context": transcription,
        "question": "Is reading papers a good idea?"
    })
except Exception as e:
    print(e)

Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 47046 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


# Splitting the Transcription

In [20]:
from langchain_community.document_loaders import TextLoader

In [21]:
loader = TextLoader("transcription.txt")

text_documents = loader.load()

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1_000, chunk_overlap=25)
documents = text_splitter.split_documents(text_documents)

In [24]:
for idx, chunk in enumerate(documents[:3], 1):
    print(f" CHUNK {idx} ".center(100, "#"))
    print(chunk.page_content)

############################################# CHUNK 1 ##############################################
I think it's possible that physics has exploits and we should be trying to find them. arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow, somehow gives you a rounding error in the floating point. Synthetic intelligences are kind of like the next stage of development. And I don't know where it leads to. Like at some point, I suspect the universe is some kind of a puzzle. These synthetic AIs will uncover that puzzle and solve it. The following is a conversation with Andre Kappathi, previously the director of AI at Tesla. And before that, at OpenAI and Stanford, he is one of the greatest scientist engineers and educators in the history of artificial intelligence. This is the Lex Friedman podcast to support it. Please check out our sponsors and now to your friends. Here's Andre Kappathi. What is a neural network? And what does it seem to do such 

# Finding the Relevant Chunks

...

# A Dummy Example

In [25]:
from langchain_openai.embeddings import OpenAIEmbeddings

In [26]:
embeddings = OpenAIEmbeddings()
embedded_query = embeddings.embed_query("Whos is Mary's sister?")

print(f"Embedding length: {len(embedded_query)}")
print(embedded_query[:10])

Embedding length: 1536
[-0.003834682982414961, -0.03383693844079971, -0.008573892526328564, 0.0023887145798653364, -0.028741013258695602, 0.006994156166911125, -0.014434205368161201, -0.003997115418314934, -0.01886766031384468, -0.030932260677218437]


In [27]:
sentence1 = embeddings.embed_query("Mary's sister is Susana")
sentence2 = embeddings.embed_query("Pedro's mother is a teacher")

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(0.9143822019773717, 0.765582207739584)

# Setting Up a Vector Store

In [30]:
%pip install -qU docarray

Note: you may need to restart the kernel to use updated packages.


In [31]:
from langchain_community.vectorstores import DocArrayInMemorySearch

In [32]:
vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "Mary's sister is Susana",
        "John and Tommy are brothers",
        "Patricia likes white cars",
        "Pedro's mother is a teacher",
        "Lucia drives an Audi",
        "Mary has two siblings",      
    ],
    embedding = embeddings,
)

/home/martin/Documents/LangChainDoc/.venv/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [33]:
vectorstore1.similarity_search_with_score(query="Whos is Mary's sister?", k=3)

[(Document(page_content="Mary's sister is Susana"), 0.9143822105391518),
 (Document(page_content='Mary has two siblings'), 0.9006237913279054),
 (Document(page_content='John and Tommy are brothers'), 0.8023114430485078)]

# Connecting the Vector Store to the Chain

In [34]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("Who is Mary's sister?")

[Document(page_content="Mary's sister is Susana"),
 Document(page_content='Mary has two siblings'),
 Document(page_content='John and Tommy are brothers'),
 Document(page_content="Pedro's mother is a teacher")]

In [35]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [37]:
setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup

{
  context: VectorStoreRetriever(tags=['DocArrayInMemorySearch'], vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x7027575dabd0>),
  question: RunnablePassthrough()
}

In [38]:
setup.invoke("What color is Patricia's car?")

{'context': [Document(page_content='Patricia likes white cars'),
  Document(page_content='Lucia drives an Audi'),
  Document(page_content="Pedro's mother is a teacher"),
  Document(page_content="Mary's sister is Susana")],
 'question': "What color is Patricia's car?"}

In [39]:
chain = setup | prompt | model | parser
chain.invoke("What color is Patricia's car?")

'White'

In [40]:
# Another example
chain.invoke("What car does Lucia drive?")

'Lucia drives an Audi.'

# Loading the Transcription into the Vector Store

In [41]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

In [42]:
chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain

{
  context: VectorStoreRetriever(tags=['DocArrayInMemorySearch'], vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x70271bf53950>),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nAnswer the question based on the context below. If you can\'t answer the question, reply "I don\'t know".\n\nContext: {context}\n\nQuestion: {question}\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x702832cac410>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x702832786790>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [43]:
chain.invoke("What is synthetic intelligence?")

'Synthetic intelligence is described as the next stage of development and is believed to be capable of uncovering and solving puzzles in the universe.'

# Setting Up Pinecone

## Infos to Retrieve

In [44]:
embeddings.model

'text-embedding-ada-002'

## Implementation

In [45]:
from langchain_pinecone import PineconeVectorStore

In [46]:
index_name = "whisper-index"

pinecone = PineconeVectorStore.from_documents(
    documents,
    embeddings,
    index_name = index_name
)

In [47]:
pinecone

In [48]:
pinecone.similarity_search("What is Hollywood going to start doing?")[:3]

[Document(page_content="well. So I don't know. Apotgas is pretty simple. It's like high quality audio and you're speaking usually pretty clearly. I don't know what open AI's plans are either. Yeah, there's always fun projects basically. And stable diffusion also is opening up a huge amount of experimentation. I would say in the visual realm and generating images and videos and movies. I'll think like videos now. And so that's going to be pretty crazy. That's going to almost certainly work and it's going to be really interesting when the cost of content creation is going to fall to zero. You used to need a painter for a few months to paint a thing and now it's going to be speak to your phone to get your video. So Hollywood will start using it to generate scenes, which completely opens up. Yeah, so you can make a movie like Avatar eventually for under a million dollars. Much less. Maybe just by talking to your phone. I mean, I know it sounds kind of crazy. And then there'd be some voting

In [49]:
chain = (
    {"context": pinecone.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("What is Hollywood going to start doing?")

'Hollywood is going to start using AI to generate scenes for movies.'